In [168]:
import matplotlib.pyplot as plt
%matplotlib inline

In [221]:
import time
import datetime
import pickle

In [170]:
import pandas as pd
import numpy as np

In [171]:
epa = pd.read_csv('../datasets/epa_violations.csv')

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (8,15) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [172]:
ticker = pd.read_csv('../datasets/market_cap_with_ticker.csv')

In [132]:
ticker.head()

,Ticker,Company Valuation (Millions)
0,GLW,23930.0
1,HPQ,29610.0
2,VZ,237970.0
3,KO,219280.0
4,FDX,40870.0


In [173]:
epa.head()

,Company,Parent Company,Penalty Amount,Subtraction From Penalty,Penalty Amount Adjusted For Eliminating Multiple Counting,Penalty Year,Penalty Date,Primary Offense,Secondary Offense,Description,...,Ownership Structure,Parent Company Stock Ticker,Major Industry of Parent,Specific Industry of Parent,Info Source,Info Source Archive URL,PACER Link,Captured Document,EPA/OSHA More Info Link,Notes
0,5N Plus Inc.,NaN,"$61,150",$0,"$61,150",2013,20130328,environmental violation,NaN,"A Fairfield, Conn. company that develops and p...",...,NaN,NaN,NaN,NaN,http://yosemite.epa.gov/opa/admpress.nsf/0c0af...,screenshot_99713.png,NaN,NaN,NaN,NaN
1,A&L Cesspool Service Corporation,NaN,"$900,000",$0,"$900,000",2016,20161026,environmental violation,NaN,A&L Cesspool Service Corporation pled guilty a...,...,NaN,NaN,NaN,NaN,https://cfpub.epa.gov/compliance/criminal_pros...,chunk_1screenshot_734000_6334526091881956598.png,NaN,NaN,NaN,NaN
2,A-1 Barrel,NaN,"$15,125",$0,"$15,125",2012,20120911,environmental violation,NaN,"A-1 Barrel Company, LLC is a used drum washing...",...,NaN,NaN,NaN,NaN,http://cfpub.epa.gov/compliance/criminal_prose...,screenshot_100029.png,NaN,NaN,NaN,NaN
3,"A-1 Petroleum and Propane, Inc.",NaN,"$40,000",$0,"$40,000",2015,20151001,environmental violation,NaN,"A-1 Petroleum and Propane, Inc., located in El...",...,NaN,NaN,NaN,NaN,http://yosemite.epa.gov/opa/admpress.nsf/21b89...,screenshot_100095.png,NaN,NaN,NaN,NaN
4,"A-1 Restaurant Services, L.L.C.",NaN,"$200,000",$0,"$200,000",2016,20151119,environmental violation,NaN,NaN,...,NaN,NaN,NaN,NaN,https://cfpub.epa.gov/compliance/criminal_pros...,screenshot_194821.png,NaN,NaN,NaN,NaN


In [174]:
cols = [col for col in epa.columns]
penalty_amt = 'Penalty Amount Adjusted For Eliminating Multiple Counting'

# create a new row with penalty amount as int
penalty_int = [int(entry.replace('$', '').replace(',','')) for entry in epa[penalty_amt]]
epa['penalty_amt'] = penalty_int

In [175]:
cols_of_interest = ['penalty_amt', 'Company', 'Parent Company', 'Parent Company Stock Ticker', 'Penalty Year', 'Penalty Date']

In [176]:
# create a new dataframe only with the above columns
epa_cols_interest = epa[cols_of_interest].copy()

In [177]:
# drop NaN rows and reindex our table
epa_cols_interest = epa_cols_interest.dropna()
epa_cols_interest = epa_cols_interest.reset_index()

In [178]:
# add market cap to epa_cols_interest
ticker_only = epa_cols_interest['Parent Company Stock Ticker']
market_cap = []
for row in ticker_only:
    market_cap.append((row, list(ticker[ticker['Ticker'] == row]['Company Valuation (Millions)'].values)))

In [179]:
m_cap = [m_c[0] if m_c else np.nan for t,m_c in market_cap]

In [180]:
epa_cols_interest['m_cap'] = m_cap

In [181]:
epa_cols_interest.head()

,index,penalty_amt,Company,Parent Company,Parent Company Stock Ticker,Penalty Year,Penalty Date,m_cap
0,11,136800,Advanced Sterilization Products,Johnson & Johnson,JNJ,2014,20140520,362890.0
1,14,140000,"AES Thames, LLC",AES Corp.,AES,2010,20100303,NaN
2,17,0,Agrifos Fertilizer LLC,Rentech,RTKH,2011,20111207,NaN
3,18,385000,Agrifos Fertilizer LLC,Rentech,RTKH,2013,20130102,NaN
4,25,1485000,Air Products LLC,Air Products & Chemicals,APD,2010,20100826,48760.0


In [182]:
epa_cols_interest = epa_cols_interest.dropna()

In [183]:
epa_cols_interest.head()

,index,penalty_amt,Company,Parent Company,Parent Company Stock Ticker,Penalty Year,Penalty Date,m_cap
0,11,136800,Advanced Sterilization Products,Johnson & Johnson,JNJ,2014,20140520,362890.0
4,25,1485000,Air Products LLC,Air Products & Chemicals,APD,2010,20100826,48760.0
5,26,0,"Air Products and Chemicals, Inc.",Air Products & Chemicals,APD,2011,20110228,48760.0
16,47,1058000,ALLTEL Corporation,Verizon Communications,VZ,2003,20031002,237970.0
19,61,9050000,Anadarko Petroleum Co.,Anadarko Petroleum,APC,2009,20090507,35230.0


In [185]:
# add new column with ratio r = m_cap / penalty_amt
r_val = []
for row in epa_cols_interest.iterrows():
    penalty = row[1][1]
    cap = row[1][-1]*10e5
    #print(penalty, cap)
    r_val.append(penalty/cap)

In [186]:
# append column to dataframe
epa_cols_interest['r_val'] = r_val

In [190]:
# sort by highest fine
# epa_cols_interest.sort_values(by=['Parent Company Stock Ticker'], ascending=False)
top_10 = epa_cols_interest.sort_values(by=['r_val'], ascending=False).head(10)

In [223]:
# Put violations into (ticker, date1, date2, amount)
def to_unix(t):
    t = str(t)
    add_slash = lambda x: x[:4]+'/'+x[4:6]+'/'+x[6:]
    t = add_slash(t)
    new_t = time.mktime(datetime.datetime.strptime(t, "%Y/%m/%d").timetuple())
    return new_t


def time_range(d,delta):
    yr_to_sec = lambda delta: delta*24*60*60*365
    prior = d - yr_to_sec(delta)
    post = d + yr_to_sec(delta)
    return (round(prior), round(post))

# are we using market cap here?
def date_range(df):
    for row in df.iterrows():
        #print(row[1])
        data = row[1]
        date = data[6]
        ticker = data[4]
        penalty = data[1]
        unix_date = to_unix(date)
        yield (ticker, date, *time_range(unix_date, 2.5), penalty)

In [224]:
test = list(date_range(top_10))
for entry in test:
    print(entry)

('APC', 20150123, 1343160000, 1500840000, 5150000000)
('NBL', 20150422, 1350846000, 1508526000, 73450000)
('HES', 20120425, 1256497200, 1414177200, 45850000)
('XOM', 20051011, 1050174000, 1207854000, 589400000)
('COP', 20011220, 929995200, 1087675200, 116500000)
('CVX', 20031016, 987447600, 1145127600, 278500000)
('CMI', 20061101, 1083528000, 1241208000, 28120000)
('XOM', 20171031, 1430593200, 1588273200, 302500000)
('CVX', 20181024, 1461524400, 1619204400, 152950000)
('HON', 20130327, 1285527600, 1443207600, 69000000)


In [225]:
# save this data in a pickle file
pickle.dump(test, open("top_r_vals.p", "wb"))

In [ ]:
# TODO:
#   split fines by amount
# Fines here are split by billions, millions, thousands, and hundreds

fines_bill = epa_cols_interest[epa_cols_interest['penalty_amt'] > 999999999]
fines_mill = epa_cols_interest[epa_cols_interest['penalty_amt'].between(999999, 999999999)]
fines_thou = epa_cols_interest[epa_cols_interest['penalty_amt'].between(999, 999999)]
fines_hund = epa_cols_interest[epa_cols_interest['penalty_amt'].between(0, 998, inclusive=True)]

In [ ]:
def plot_years_sum(df):
    fine_by_year = []
    unique_years = df['Penalty Year'].unique()
    for year in unique_years:
        epa_years = df[df['Penalty Year'] == year]
        fine_by_year += [(sum(epa_years['penalty_amt']), year)]
        
    sorted_fines = sorted(fine_by_year, key = lambda x: x[1])
    x_axis = [year for _,year in sorted_fines]
    y_axis = [fine for fine,_ in sorted_fines]
    
    plt.plot(x_axis, y_axis)
    plt.title("Fines by Year")
    plt.show()

In [ ]:
plot_years_sum(fines_bill)
plot_years_sum(fines_mill)
plot_years_sum(fines_thou)
plot_years_sum(fines_hund)
plot_years_sum(epa_cols_interest)

In [ ]:
# TOP FINES BY YEAR (using main dataframe epa_cols_interest)
unique_years = sorted(epa_cols_interest['Penalty Year'].unique())
for year in unique_years:
    this_year = epa_cols_interest[epa_cols_interest['Penalty Year'] == year]
    penalty = list(this_year['penalty_amt'])
    ticker = list(this_year['Parent Company Stock Ticker'])
    max_ = max(zip(penalty, ticker))
    print(year, *max_)

<h1> not really running this </h1>

In [ ]:
sd = epa_cols_interest['penalty_amt'].std()
avg = epa_cols_interest['penalty_amt'].mean()

In [ ]:
#normalize values
z_score = lambda x: ((x - avg) / float(sd))

epa_cols_interest['normalized_amt'] = [z_score(val) for val in epa_cols_interest['penalty_amt']]

In [ ]:
test_set = sorted(epa_cols_interest['normalized_amt'])[::-1]
test_set

plt.plot(test_set)
plt.show()